# Tiny Stories Analysis Write Up

## Setup

In [1]:
import sys 
sys.path.append("..")

from importlib import reload

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)


/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:


model = HookedTransformer.from_pretrained(
    "tiny-stories-2L-33M",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
# model.set_use_split_qkv_input(True)
# model.set_use_attn_result(True)
import transformer_lens.evals as evals
evals.sanity_check(model)

path = "checkpoints/399ihu5z/final_sparse_autoencoder_tiny-stories-2L-33M_blocks.1.attn.hook_q_4096.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import itertools
import numpy as np

def tensor_to_long_data_frame(tensor_result, dimension_names):
    assert len(tensor_result.shape) == len(
        dimension_names
    ), "The number of dimension names must match the number of dimensions in the tensor"

    tensor_2d = tensor_result.reshape(-1)
    df = pd.DataFrame(tensor_2d.detach().numpy(), columns=["Score"])

    indices = pd.MultiIndex.from_tuples(
        list(np.ndindex(tensor_result.shape)),
        names=dimension_names,
    )
    df.index = indices
    df.reset_index(inplace=True)
    return df

q_composition_scores = model.all_composition_scores(mode = "Q")
q_comp_df = tensor_to_long_data_frame(q_composition_scores.detach().cpu(), ["Layer1", "Head1", "Layer2", "Head2"])
q_comp_df.sort_values("Score", ascending=False).head(10).style.background_gradient(cmap="Blues")

,Layer1,Head1,Layer2,Head2,Score
219,0,6,1,11,0.100765
211,0,6,1,3,0.080608
218,0,6,1,10,0.066766
408,0,12,1,8,0.060902
208,0,6,1,0,0.060876
213,0,6,1,5,0.059937
401,0,12,1,1,0.059086
403,0,12,1,3,0.058306
146,0,4,1,2,0.056562
150,0,4,1,6,0.056216


In [ ]:
def get_webtext(seed: int = 420, dataset="stas/openwebtext-10k", split="train[:1%]") -> List[str]:
    """Get 10,000 sentences from the OpenWebText dataset"""

    # Let's see some WEBTEXT
    train_dataset = load_dataset(dataset, split=split)
    # train_dataset = raw_dataset["train"]
    dataset = [train_dataset[i]["text"] for i in range(len(train_dataset))]

    # Shuffle the dataset (I don't want the Hitler thing being first so use a seeded shuffle)
    np.random.seed(seed)
    np.random.shuffle(dataset)

    return dataset

data = get_webtext(dataset="roneneldan/TinyStories")

In [ ]:
from tqdm import tqdm

str_token_list = []
loss_list = []
ablated_loss_list = []

import joseph.analysis as analysis
reload(analysis)

NUM_PROMPTS = 200
# MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
with torch.no_grad():
    model.reset_hooks()
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = data[i]
        # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
        token_df, _, _= analysis.eval_prompt(prompt, model=model, sparse_autoencoder=sparse_autoencoder)
        token_df["batch"] = i
        token_df["label"] = token_df["batch"].astype(str) + "/" + token_df["pos"].astype(str)
        dataframe_list.append(token_df)
        
head_analysis_df = pd.concat(dataframe_list)

print(head_analysis_df.columns)



 29%|██▉       | 58/200 [00:42<01:48,  1.31it/s]

: 

In [ ]:
print(head_analysis_df.shape)

In [ ]:
filter_cols = ["unique_token", "context", "label", "loss", "loss_diff", 
               "mse_loss", "num_active_features", "max_idx_tok", "rec_q_max_idx_tok",
               "explained_variance", "top10_token_suppression_inds","top10_token_boosting_inds"]

tmp = head_analysis_df[filter_cols].sort_values("loss_diff", ascending=False)
tmp.head(10).reset_index().style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features"],# "top10_token_suppression_diffs", "top10_token_suppression_inds"],
    cmap="coolwarm")

In [ ]:
head_analysis_df["max_idx_tok_suppressed"] = head_analysis_df.apply(lambda x: x["max_idx_tok"] in x["top10_token_suppression_inds"], axis=1)
head_analysis_df["rec_q_max_idx_tok_suppressed"] = head_analysis_df.apply(lambda x: x["rec_q_max_idx_tok"] in x["top10_token_suppression_inds"], axis=1)
print(head_analysis_df["max_idx_tok_suppressed"].mean())
print(head_analysis_df["rec_q_max_idx_tok_suppressed"].mean())

head_analysis_df["max_idx_tok_boosted"] = head_analysis_df.apply(lambda x: x["max_idx_tok"] in x["top10_token_boosting_inds"], axis=1)
head_analysis_df["rec_q_max_idx_tok_boosted"] = head_analysis_df.apply(lambda x: x["rec_q_max_idx_tok"] in x["top10_token_boosting_inds"], axis=1)
print(head_analysis_df["max_idx_tok_boosted"].mean())
print(head_analysis_df["rec_q_max_idx_tok_boosted"].mean())

px.bar(pd.DataFrame( {"Most Attended Token is Suppressed": head_analysis_df["max_idx_tok_suppressed"].mean(),
        "Most Attended Token is Boosted": head_analysis_df["max_idx_tok_boosted"].mean(),
        "Most Attended Token is Suppressed (Rec Q)": head_analysis_df["rec_q_max_idx_tok_suppressed"].mean(),
        "Most Attended Token is Boosted (Rec Q)": head_analysis_df["rec_q_max_idx_tok_boosted"].mean(),
    }, index=[0]).T
)


In [ ]:
px.strip(head_analysis_df, x="loss_diff", color="max_idx_tok_suppressed", hover_data=["context", "max_idx_tok", "top10_token_suppression_inds"]).show()
px.strip(head_analysis_df, x="loss_diff", color="max_idx_tok_boosted", hover_data=["context", "max_idx_tok", "top10_token_boosting_inds"]).show()

In [ ]:
filter_cols = ["str_tokens", "label","context", "loss_diff", "max_idx_tok", "rec_q_max_idx_tok", "top10_token_suppression_inds","top_k_features"]
head_analysis_df[head_analysis_df.max_idx_tok_suppressed][filter_cols].sample(30).sort_values("loss_diff", ascending=True).reset_index()

In [ ]:
# do these features have high dot product with "," unembed?
token = ","
tmp = head_analysis_df[head_analysis_df.max_idx_tok_suppressed]
tmp = tmp[tmp.max_idx_tok == token]
top_k_features_for_comma = tmp.explode("top_k_features").top_k_features.value_counts()[:2]
features = top_k_features_for_comma.index
token_id = model.to_single_token(token)
query_feature_unembed_virtual_weight = sparse_autoencoder.W_dec @ model.W_Q[LAYER_IDX,HEAD_IDX].T @ model.W_U[:,token_id]
px.bar(x = [str(i) for i in range(4096)], y =query_feature_unembed_virtual_weight.detach().cpu()).show()
random_features = torch.randint(0, 4096, (1000,))
random_feature_unembed_virtual_weight = sparse_autoencoder.W_dec[random_features] @ model.W_Q[LAYER_IDX,HEAD_IDX].T @ model.W_U[:,token_id]
px.histogram(random_feature_unembed_virtual_weight.flatten().detach().cpu(), log_y=True).show()

In [ ]:
all_virtual_weights = sparse_autoencoder.W_dec @ model.W_Q[LAYER_IDX,HEAD_IDX].T @ model.W_U
torch.quantile(all_virtual_weights[all_virtual_weights > 0.05], q = 0.99999)

Looking at diagrams:
- we can see that 1949 appears to be genuinely a "," is next feature (though it fires often for "Suddenly").
- whilst 474 might be slightly more a ". Then" (two token?) feature. 
- 3619 looks like a dense feature so my guess is it has positive high average scores
- 2722 is often "Day" (often appears before a ","). Lower activations for things like birthday.

What this means is that when "," is often suppressed by this head but can the SAE splits that up into token specific situations. 
Let's check the cosine sim on the encoder/decoder weights.

It seems like 474 and 1949 which should be pretty mutually exclusive might be somewhat confused with each other due to both being partly about inhibiting ",".


Can we say that a feature specifies given a tokem, whether or not we are doing suppression?

In [ ]:
token = ","
tmp = head_analysis_df.query(f"max_idx_tok == '{token}'")
filter_cols = ["unique_token", "context", "label", "loss_diff", "max_idx_tok", "rec_q_max_idx_tok", "top10_token_suppression_inds","max_idx_tok_suppressed", "max_idx_tok_value"]
print(tmp.max_idx_tok_suppressed.mean())
tmp[filter_cols].sort_values("max_idx_tok_value", ascending=False).reset_index()

In [ ]:

tmp = head_analysis_df[head_analysis_df.max_idx_tok_suppressed]
tmp = tmp[tmp.max_idx_tok == token]
tmp[filter_cols].sample(30).sort_values("loss_diff", ascending=True).reset_index().style.background_gradient(
    subset=["loss_diff", "num_active_features"],# "top10_token_suppression_diffs", "top10_token_suppression_inds"],
    cmap="coolwarm")

top_k_features_for_comma = tmp.explode("top_k_features").top_k_features.value_counts()[:2]
top_k_features_for_comma

In [ ]:
feature = 3506; token = token
feature_token_mask = head_analysis_df.query(f"max_idx_tok == '{token}'").top_k_features.apply(lambda x: feature in x)
tmp = head_analysis_df.query(f"max_idx_tok == '{token}'")[feature_token_mask]
px.histogram(tmp.apply(lambda x: x["top_k_feature_acts"][x["top_k_features"].index(feature)], axis =1)).show()
filter_cols = ["unique_token", "context", "label", "loss_diff", "max_idx_tok", "rec_q_max_idx_tok", "top10_token_suppression_inds","max_idx_tok_suppressed", "max_idx_tok_value"]
tmp[filter_cols].sort_values("max_idx_tok_value", ascending=False).reset_index()

In [ ]:
# set pandas width unlimited
pd.set_option('display.max_colwidth', None)

filter_cols = ["unique_token", "context", "label", "loss_diff", "max_idx_tok", "rec_q_max_idx_tok", "top10_token_suppression_inds","max_idx_tok_suppressed"]

top_k_features_for_comma = tmp.explode("top_k_features").top_k_features.value_counts()[:10].index
comma_attended_to_set = head_analysis_df.query("max_idx_tok == ','")
comma_not_attended_to_set = head_analysis_df.query("max_idx_tok != ','")
for i in top_k_features_for_comma:
    print(i)
    print("feature prob given token", comma_attended_to_set.top_k_features.apply(lambda x: i in x).mean())
    print("feature prob given not token", comma_not_attended_to_set.top_k_features.apply(lambda x: i in x).mean())
    print("ratio prob:", comma_attended_to_set.top_k_features.apply(lambda x: i in x).mean() / comma_not_attended_to_set.top_k_features.apply(lambda x: i in x).mean())
    print("prob max idx suppressed given token", comma_attended_to_set[comma_attended_to_set.top_k_features.apply(lambda x: i in x)].max_idx_tok_suppressed.mean())
    print("prob max idx suppressed given not token", comma_not_attended_to_set[comma_not_attended_to_set.top_k_features.apply(lambda x: i in x)].max_idx_tok_suppressed.mean())
    display(comma_attended_to_set[comma_attended_to_set.top_k_features.apply(lambda x: i in x)].sort_values(
        "max_idx_tok_value").head(10)[filter_cols])
    display(comma_not_attended_to_set[comma_not_attended_to_set.top_k_features.apply(lambda x: i in x)].sort_values(
        "max_idx_tok_value").head(10)[filter_cols])
    print("------")

In [ ]:
plot_qk_via_feature(model, 2722, sparse_autoencoder, feature_name = "", highlight_tokens=[","])

In [ ]:
from scipy.cluster import hierarchy

def plot_cosine_similarity_heatmap(df, restricted_labels, reorder = False, title="Pairwise Cosine Similarity Heatmap"):
    data_array = df.to_numpy()
    linkage = hierarchy.linkage(data_array)
    dendrogram = hierarchy.dendrogram(
        linkage, no_plot=True, color_threshold=-np.inf
    )
    if reorder:
        reordered_ind = dendrogram["leaves"]
        # reorder df by ind
        df = df.iloc[reordered_ind, reordered_ind]
        # data_array = df.to_numpy()

    # plot the cosine similarity matrix
    fig = fig = px.imshow(
            df,
            color_continuous_scale="RdBu",
            color_continuous_midpoint=0.0,
            labels={"color": "Cosine Similarity"},
        )
    fig.update_xaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )
    fig.update_yaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )

    # don't show axes if there are more than 20 rows 
    if df.shape[0] > 20:
        fig.update_xaxes(
            visible=False,
        )
        fig.update_yaxes(
            visible=False,
        )
    return fig

feature_dirs = pd.DataFrame(sparse_autoencoder.W_dec[features].detach().cpu().numpy(), index= features).reset_index(drop=True).T.corr()
plot_cosine_similarity_heatmap(feature_dirs, restricted_labels=features).show()
feature_dirs = pd.DataFrame(sparse_autoencoder.W_enc[:,features].T.detach().cpu().numpy(), index= features).reset_index(drop=True).T.corr()
plot_cosine_similarity_heatmap(feature_dirs, restricted_labels=features).show()

Last thing to do before writing up the report:
- Show that these features are doing copy suppression causally. 

In [ ]:
plot_qk_via_feature(model, 1949, sparse_autoencoder, feature_name = "", highlight_tokens=[","])
# plot_qk_via_feature(model, 3102, sparse_autoencoder, feature_name = "", highlight_tokens=[" Lily"])
# plot_qk_via_feature(model, 2656, sparse_autoencoder, feature_name = "", highlight_tokens=[" Lily"])

In [ ]:
batch = 118; pos = 94
# batch = 48; pos = 15
prompt = data[batch]
tokens = model.to_tokens(prompt)
example_prompt = model.to_string(tokens[:,:pos+1])[0]
example_prompt_answer = model.to_string(tokens[:,pos+1])
# example_prompt_answer = " Lily"
example_prompt_tokens = tokens[:,:pos]
_, cache_original = model.run_with_cache(example_prompt_tokens, prepend_bos=False)

def test_prompt_with_sae(example_prompt, example_prompt_answer, model, sparse_autoencoder, cache_original):
    
    model.reset_hooks()
    utils.test_prompt(example_prompt, example_prompt_answer, model, prepend_space_to_answer=False, prepend_bos=False)

    with model.hooks(fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)]):
        utils.test_prompt(example_prompt, example_prompt_answer, model, prepend_bos=False, prepend_space_to_answer=False)

    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        cache_original[sparse_autoencoder.cfg.hook_point][0,HEAD_IDX]
    )
    def reconstr_query_hook(hook_in, hook, reconstructed_query=sae_out, head = HEAD_IDX):
        hook_in[:, head, :] = reconstructed_query
        return hook_in

    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, reconstr_query_hook)]):
        utils.test_prompt(example_prompt, example_prompt_answer, model,  prepend_bos=False, prepend_space_to_answer=False)
        
    
test_prompt_with_sae(example_prompt, example_prompt_answer, model, sparse_autoencoder, cache_original)


In [ ]:
model.reset_hooks()
token_df, original_cache, cache_reconstructed_query = eval_prompt([example_prompt], model, sparse_autoencoder)

In [ ]:
print(prompt)

In [ ]:
original_acts = original_cache[sparse_autoencoder.cfg.hook_point]
print(original_acts.shape)
sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
    original_cache[sparse_autoencoder.cfg.hook_point][:,-1,HEAD_IDX]
)
print(feature_acts.shape)
vals, inds = torch.topk(feature_acts[0], 10)
print(inds)
plot_line_with_top_10_labels(feature_acts[0])

In [ ]:
 model.W_Q[LAYER_IDX,HEAD_IDX].shape

In [ ]:
def test_get_attn_sae_feature_removal(example_prompt, 
                                      example_prompt_answer, 
                                      model, 
                                      sparse_autoencoder,
                                      features_to_remove=[0],
                                      token_unembed_to_remove=[]):
    
    
    example_prompt_tokens = model.to_tokens(example_prompt)
    attn_df, original_cache, cache_reconstructed_query = eval_prompt([example_prompt], model, sparse_autoencoder)

    original_acts = original_cache[sparse_autoencoder.cfg.hook_point]
    print(original_acts.shape)
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
       original_acts[0,:,HEAD_IDX]
    )
    

    def remove_feature_hook(hook_in, hook, head = HEAD_IDX, features_to_remove = features_to_remove):
        for feature_to_remove in features_to_remove:
            feature_dir = feature_acts[-1,feature_to_remove]*sparse_autoencoder.W_dec[feature_to_remove]
            hook_in[:, :, head] -= feature_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]):
        _, cache_removed_feature = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        
        
    def replace_with_feature_hook(hook_in, hook, head = HEAD_IDX, features_to_add = features_to_remove):
        new_act = torch.zeros_like(hook_in[:, :, head])
        for features_to_add in features_to_remove:
            feature_dir = feature_acts[-1,features_to_add]*sparse_autoencoder.W_dec[features_to_add]
            new_act += feature_dir
        hook_in[:, :, head] = new_act
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, replace_with_feature_hook)]):
        _, cache_only_features = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        

    def remove_token_unembed_hook(hook_in, hook, head = HEAD_IDX, token_unembed_to_remove = token_unembed_to_remove):
        for token_id in token_unembed_to_remove:
            unembed_dir = model.W_U[:, token_id] @ model.W_Q[LAYER_IDX,HEAD_IDX]
            unembed_dir_proj =  hook_in[:, :, head] @ unembed_dir.T
            hook_in[:, :, head] -= unembed_dir_proj[:,:,None]*unembed_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_token_unembed_hook)]):
        _, cache_removed_token_unembed = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)

    patterns = original_cache[f"blocks.{LAYER_IDX}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df = make_token_df(model, example_prompt_tokens)
    # remove last row of attn df
    attn_df["original_attn"] = patterns[-1,]
    patterns = cache_reconstructed_query[f"blocks.{LAYER_IDX}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df["reconstructed_attn"] = patterns[-1,]
    patterns = cache_removed_feature[f"blocks.{LAYER_IDX}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df["ablated_feature_attn"] = patterns[-1,]
    patterns = cache_only_features[f"blocks.{LAYER_IDX}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df["cache_only_features"] = patterns[-1,]
    patterns = cache_removed_token_unembed[f"blocks.{LAYER_IDX}.attn.hook_pattern"][0,HEAD_IDX].detach().cpu()
    attn_df["cache_removed_token_unembed"] = patterns[-1,]
    
    if len(token_unembed_to_remove) > 0:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features", "cache_removed_token_unembed"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    else:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    
    # increase figure height
    fig.update_layout(height=800)
    fig.show()

def test_prompt_with_sae_feature_removal(example_prompt, example_prompt_answer, model, sparse_autoencoder, features_to_remove=[0]):
    
    model.reset_hooks()
    print("ORIGINAL")
    utils.test_prompt(example_prompt, example_prompt_answer, model, prepend_space_to_answer=False, prepend_bos=False)

    print("HEAD ABLATED")
    with model.hooks(fwd_hooks=[(HEAD_HOOK_RESULT_NAME, hook_to_ablate_head)]):
        utils.test_prompt(example_prompt, example_prompt_answer, model, prepend_bos=False, prepend_space_to_answer=False)

    token_df, original_cache, cache_reconstructed_query = eval_prompt([example_prompt], model, sparse_autoencoder)
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        original_cache[sparse_autoencoder.cfg.hook_point][0,HEAD_IDX]
    )
    def reconstr_query_hook(hook_in, hook, reconstructed_query=sae_out, head = HEAD_IDX):
        hook_in[:, head, :] = reconstructed_query
        return hook_in

    def remove_feature_hook(hook_in, hook, head = HEAD_IDX, features_to_remove = features_to_remove):
        print(hook_in.shape)
        for feature_to_remove in features_to_remove:
            feature_dir = feature_acts[-1,feature_to_remove]*sparse_autoencoder.W_dec[feature_to_remove]
            hook_in[:, :, head] -= feature_dir
        return hook_in
    
    print("FEATURE REMOVED")
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]):
        utils.test_prompt(example_prompt, example_prompt_answer, model,  prepend_bos=False, prepend_space_to_answer=False)

    def replace_with_feature_hook(hook_in, hook, head = HEAD_IDX, features_to_add = features_to_remove):
        new_act = torch.zeros_like(hook_in[:, :, head])
        for features_to_add in features_to_remove:
            feature_dir = feature_acts[-1,features_to_add]*sparse_autoencoder.W_dec[features_to_add]
            new_act += feature_dir
        hook_in[:, :, head] = new_act
        return hook_in
    
    print("ONLY FEATURE USED")
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, replace_with_feature_hook)]):
        utils.test_prompt(example_prompt, example_prompt_answer, model,  prepend_bos=False, prepend_space_to_answer=False)
        

test_get_attn_sae_feature_removal(
    example_prompt, 
    example_prompt_answer, model, 
    sparse_autoencoder, features_to_remove=[1949, 474], token_unembed_to_remove=[model.to_single_token(",")])

test_prompt_with_sae_feature_removal(
    example_prompt, 
    example_prompt_answer, model, 
    sparse_autoencoder, features_to_remove=[1949])

Great! So clearly these features are able to cause copy suppression for that token.

In [ ]:
from importlib import reload
from sae_analysis.visualizer import data_fns
from sae_analysis.visualizer.data_fns import FeatureData
from typing import Dict
reload(data_fns)


os.environ["TOKENIZERS_PARALLELISM"] = "false"
dataset="roneneldan/TinyStories"
train_dataset = load_dataset(dataset, split="train[:3%]")
tokenized_data = utils.tokenize_and_concatenate(train_dataset, model.tokenizer, max_length=128)
tokenized_data = tokenized_data.shuffle(42)
all_tokens = tokenized_data["tokens"]


# Currently, don't think much more time can be squeezed out of it. Maybe the best saving would be to
# make the entire sequence indexing parallelized, but that's possibly not worth it right now.

max_batch_size = 32
total_batch_size = 512 * 50
feature_idx = top_k_features_for_comma.index.to_list()
feature_idx = inds.tolist()
# max_batch_size = 512
# total_batch_size = 16384
# feature_idx = list(range(1000))

tokens = all_tokens[:total_batch_size]

feature_data: Dict[int, FeatureData] = data_fns.get_feature_data(
    encoder=sparse_autoencoder,
    # encoder_B=sparse_autoencoder,
    model=model,
    hook_point=sparse_autoencoder.cfg.hook_point,
    hook_point_layer=sparse_autoencoder.cfg.hook_point_layer - 1,
    hook_point_head_index=sparse_autoencoder.cfg.hook_point_head_index,
    tokens=tokens,
    feature_idx=feature_idx,
    max_batch_size=max_batch_size,
    left_hand_k = 3,
    buffer = (5, 5),
    n_groups = 10,
    first_group_size = 20,
    other_groups_size = 5,
    verbose = True,
)

for test_idx in feature_idx:
    html_str = feature_data[test_idx].get_all_html()
    path = f"tiny_stories_features/{test_idx:04}.html"
    print(f"Saving to {path}")
    with open(path, "w") as f:
        f.write(html_str)


del feature_data
torch.mps.empty_cache()

# Let's identify the features firing

# TO DO:

1. We can now track copy suppression and copying behavior. 
2. Following from that we want to check that this is real and that we can find the relevant features.
3. Given some feature we want:
   1. The enc / dec direction.
   2. The unembed of the token/s it's supposed to suppress.
   3. The virtual weight between the two. 
   4. Causal intervention to remove the unembed dir and see what effect we get

In [ ]:
del feature_data
torch.mps.empty_cache()

In [ ]:
test = pd.DataFrame({"1": [1,2,3], "2": [4,5,6], "3":[ {1:2, 2:3}, {3:4, 4:5}, {5:6, 1:3}]})
test.explode("3")